# Parent Document Retriever
- 문서를 검색하기 위해 분할할 때 상충되는 요구가 있음
- 작은 문서는 임베딩이 의미를 가장 정확하게 반영할 수 있음
- 너무 길면 임베딩이 의미를 잃을 수 있음
- 각 청크의 문맥이 유지될 수 있도록 충분히 긴 문서가 필요함
- ParentDocumentRetriever는 작은 데이터 청크를 분할하고 저장하여 균형을 맞춤
- 검색 시 먼저 작은 청크를 가져온 후 해당 청크의 부모 ID를 조회하여 더 큰 문서를 반환함
- "부모 문서"는 작은 청크가 유래한 문서를 의미하며, 전체 원시 문서 또는 더 큰 청크일 수 있음


In [6]:
from langchain.retrievers import ParentDocumentRetriever

In [7]:
from langchain.storage import InMemoryStore
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
load_dotenv('../dot.env')

True

In [8]:
loaders = [
    TextLoader("./files/train-geon-example-double_org.txt")]
docs = []
for loader in loaders:
    docs.extend(loader.load())

In [9]:
# This text splitter is used to create the child documents
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)
# The vectorstore to use to index the child chunks
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)
# The storage layer for the parent documents
store = InMemoryStore()
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store, # -> 부모문서를 저장하는 데에 사용되며, 현재는 InMemoryStore로 초기화, 아래의 shell의 add_documents 메소드로 docs추가
    child_splitter=child_splitter,
)

In [13]:
retriever.add_documents(docs, ids=None)